## Dynamic Programming

In [1]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [4]:
from collections import defaultdict
price = defaultdict(int)

In [5]:
for i, p in enumerate(original_price):
    price[i+1] = p

In [6]:
assert price[1] == 1

In [7]:
# Calculate the total revenue.
# REV = max(P[n], r(1) + r(n-1), r(2) + r(n-2), ..., r(n-1)+ r(1))

def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [9]:
r(4)

10

In [11]:
# r(121)

# This really takes a looooong time!
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)


In [98]:
solution = {}
## For a given length N, we want to find an optimal splitting pattern:
## eg: solution = {
##             4: (2, 2)
##    }

In [99]:
# Update func r(n)


def r(n):
    max_price, split_point =  max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key = lambda x: x[0]
    )
    
    solution[n] = (split_point, n - split_point) # Append split solutions to dictionary
    
    return max_price


In [95]:
r(15)

43

In [100]:
r(5)

13

In [101]:
solution

{1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (2, 2), 5: (2, 3)}

In [102]:
# Figuring out the result takes long time because our program 
# did a lot of repetitive calculations that had done but not saved.
# In python 3+, use decorator lru_cache:

from functools import lru_cache
@lru_cache(maxsize=2**10)

def r1(n):
    
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    
    return max_price



In [103]:
r1(15)

43

In [104]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13)}

### 装饰器

In [24]:
import time
def func_1(n): 
    for i in range(n):
        print(n)
    
def func_slow(n):
    for i in range(n):
        time.sleep(0.15)
        print(n)

In [25]:
# X Oriented --- X: 1. as arg 2. as variable(able to assign an value) 
#                   3. return
# Functions in python can be used as args of another function.

def call_time(func_1, arg): #脚手架程序
    start = time.time()
    func_1(arg)
    print('Time used: {}'.format(time.time() - start))

In [47]:
# Due to the fuction oriented feature of python, 
# we can wrap a function to form a new function.

def get_call_time(func):
    
    def _wrap(arg):
        """ It's a inner function."""
        start = time.time()
        result = func(arg)
        print('Time used: {}'.format(time.time() - start))
        return result
    return _wrap

In [27]:
call_time(func_slow, 10)

10
10
10
10
10
10
10
10
10
10
Time used: 1.5191559791564941


In [30]:
get_time_func_slow = get_call_time(func_slow)

In [32]:
get_time_func_slow.__name__

'_wrap'

In [33]:
assert get_time_func_slow(10) == call_time(func_slow, 10)

10
10
10
10
10
10
10
10
10
10
Time used: 1.5317811965942383
10
10
10
10
10
10
10
10
10
10
Time used: 1.5205345153808594


In [34]:
# Update func_slow.
func_slow = get_call_time(func_slow) ## => @ (decorator)
func_slow(10)
# Pass!

10
10
10
10
10
10
10
10
10
10
Time used: 1.5138325691223145


In [38]:
## See how a decorator works!
## No need to modify the original fuctions while adding a new feature
@get_call_time
def func_1(n): 
    for i in range(n):
        print(n)

In [40]:
func_1(5)

5
5
5
5
5
Time used: 0.0009992122650146484


In [41]:
def func_1(n): 
    for i in range(n):
        print(n)
func_1(5)
# No time info here.

5
5
5
5
5


In [ ]:
# Count function called time.
function_called_time = defaultdict(int)

def get_called_time(func):
    def _inner(arg): 
        ## (*args, **kwargs) => support all args, keyword args
        
        global function_called_time
        function_called_time[func.__name__] += 1
        
        result = func(arg)
        print('Function called time is: {}'.format(function_called_time[func.__name__]))
        return result
    return _inner


Decorators may cause a loss of important docs inside a function. See as follows:

In [42]:
def func_x(n): 
    """
    @param n: the number of customers
     
    """
    for i in range(n):
        print(n)
    return 0


In [43]:
help(func_x)

Help on function func_x in module __main__:

func_x(n)
    @param n: the number of customers



In [53]:
# Add a decorator. See docs changed.
@get_call_time
def func_x(n): 
    """
    @param n: the number of customers
     
    """
    for i in range(n):
        print(n)
    return 0


In [49]:
help(func_x)

Help on function _wrap in module __main__:

_wrap(arg)
    It's a inner function.



In [52]:
# Use wraps to fix the problem.

from functools import wraps

def get_call_time(func):
    @wraps(func)
    
    def _wrap(arg):
        """ It's a inner function."""
        start = time.time()
        result = func(arg)
        print('Time used: {}'.format(time.time() - start))
        return result
    return _wrap


In [54]:
help(func_x)

# Fix it!

Help on function func_x in module __main__:

func_x(n)
    @param n: the number of customers



### 动态规划问题分析：

1. 分析问题，找到重复子问题
2. (利用装饰器) 存储子问题结果
3. 取得解决方案

## Parse Solution

In [60]:
def not_cut(split): 
    return split == 0

def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    
    if not_cut(left): 
        return [right]
    
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)


    

In [ ]:
# TypeError: _wrap() takes 1 positional argument but were given 2

## P2: Edit Distance

In [75]:
p2solution = {}

In [65]:
@lru_cache(maxsize=2**10)
# Abstract Examples:
# 1. str1--- abcde | str2 --- abcd   (str1 delete tail)
# 2. str1--- abcde | str2 --- abcdef (str1 add tail)
# 3. str1--- abcde | str2 --- abcdh  (tail substitution)
# Substitution => delete 1 char, add 1 char => total distance 2
# Base case: one string is empty.

def edit_distance(str1, str2):
    
    if len(str1) == 0: return len(str2)
    if len(str2) == 0: return len(str1)
    
    str1_tail = str1[-1]
    str2_tail = str2[-1]
    
    return min(
    [
        edit_distance(str1[:-1], str2) + 1, # str1 delete tail
        edit_distance(str1, str2[:-1]) + 1, # str2 delete tail/str1 add tail
        edit_distance(str1[:-1], str2[:-1]) + (0 if str1_tail == str2_tail else 2)
    ]
    )

In [62]:
edit_distance('beijing', 'beijingggg')

3

In [64]:
edit_distance('beijing', 'biejing')

2

In [67]:
edit_distance('我今日确实不想吃饭', '我今天真的不太想吃饭')

7

In [68]:
edit_distance('1010', '11100')

3

In [76]:
@lru_cache(maxsize=2**10)
def edit_distance(str1, str2):
    
    if len(str1) == 0: return len(str2)
    if len(str2) == 0: return len(str1)
    
    str1_tail = str1[-1]
    str2_tail = str2[-1]
    
    candidates = [
        (edit_distance(str1[:-1], str2) + 1, 'DEL {}'.format(str1_tail)), # str1 delete tail
        (edit_distance(str1, str2[:-1]) + 1, 'ADD {}'.format(str2_tail)), # str2 delete tail    
    ]
    
    if str1_tail == str2_tail:
        both_forward = (edit_distance(str1[:-1], str2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(str1[:-1], str2[:-1]) + 1, 'SUB {} => {}'.format(str1_tail, str2_tail))
        # For parsing solution purpose, make substitution distance to 1. (sorting!)
    
    candidates.append(both_forward)
   
    min_distance, operation = min(candidates, key = lambda x: x[0])
    
    p2solution[(str1, str2)] = operation
    
    return min_distance

In [77]:
edit_distance('1010', '11100')

3

In [78]:
p2solution

{('1', '1'): '',
 ('1', '11'): 'ADD 1',
 ('1', '111'): 'ADD 1',
 ('1', '1110'): 'ADD 0',
 ('1', '11100'): 'ADD 0',
 ('10', '1'): 'DEL 0',
 ('10', '11'): 'DEL 0',
 ('10', '111'): 'DEL 0',
 ('10', '1110'): '',
 ('10', '11100'): 'ADD 0',
 ('101', '1'): 'DEL 1',
 ('101', '11'): '',
 ('101', '111'): 'ADD 1',
 ('101', '1110'): 'DEL 1',
 ('101', '11100'): 'DEL 1',
 ('1010', '1'): 'DEL 0',
 ('1010', '11'): 'DEL 0',
 ('1010', '111'): 'DEL 0',
 ('1010', '1110'): '',
 ('1010', '11100'): 'ADD 0'}

In [81]:
edit_distance('ABCD', 'ABCC')

2

In [82]:
p2solution

{('1', '1'): '',
 ('1', '11'): 'ADD 1',
 ('1', '111'): 'ADD 1',
 ('1', '1110'): 'ADD 0',
 ('1', '11100'): 'ADD 0',
 ('10', '1'): 'DEL 0',
 ('10', '11'): 'DEL 0',
 ('10', '111'): 'DEL 0',
 ('10', '1110'): '',
 ('10', '11100'): 'ADD 0',
 ('101', '1'): 'DEL 1',
 ('101', '11'): '',
 ('101', '111'): 'ADD 1',
 ('101', '1110'): 'DEL 1',
 ('101', '11100'): 'DEL 1',
 ('1010', '1'): 'DEL 0',
 ('1010', '11'): 'DEL 0',
 ('1010', '111'): 'DEL 0',
 ('1010', '1110'): '',
 ('1010', '11100'): 'ADD 0',
 ('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'DEL D'}